# Phase 2 Code Challenge

This code challenge is designed to test your understanding of the Phase 2 material. It covers:

- SQL
- Bayesian Statistics
- Normal Distribution
- Statistical Tests

_Read the instructions carefully_. You will be asked both to write code and to answer short answer questions.

## Code Tests

We have provided some code tests for you to run to check that your work meets the item specifications. Passing these tests does not necessarily mean that you have gotten the item correct - there are additional hidden tests. However, if any of the tests do not pass, this tells you that your code is incorrect and needs changes to meet the specification. To determine what the issue is, read the comments in the code test cells, the error message you receive, and the item instructions.

## Short Answer Questions 

For the short answer questions...

* _Use your own words_. It is OK to refer to outside resources when crafting your response, but _do not copy text from another source_.

* _Communicate clearly_. We are not grading your writing skills, but you can only receive full credit if your teacher is able to fully understand your response. 

* _Be concise_. You should be able to answer most short answer questions in a sentence or two. Writing unnecessarily long answers increases the risk of you being unclear or saying something incorrect.

In [1]:
# Run this cell without changes to import the necessary libraries

import itertools
import numpy as np
import pandas as pd 
from numbers import Number
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

import pickle

---
## Part 1: SQL [Suggested time: 20 minutes]
---
In this part, you will create and execute three SQL queries on the Chinook database. For this challenge **you will need to access the `Album` and `Artist` tables**.

### 1.1) Connect to the Database.

In [3]:
# CodeGrade step1.1
# Replace None with appropriate code
# Connect to the Database here ("Chinook_Sqlite.sqlite")

path = 'Chinook_Sqlite.sqlite'
conn = sqlite3.connect(path)

### 1.2) Write a query to return the last ten artists alphabetically.

In [6]:
# CodeGrade step1.2
# Replace None with appropriate code
# Hint: Use the Artist table!

first_query = "SELECT * FROM Artist ORDER BY Name DESC LIMIT 10"

pd.read_sql(first_query, conn)

,ArtistId,Name
0,155,Zeca Pagodinho
1,168,Youssou N'Dour
2,212,Yo-Yo Ma
3,255,Yehudi Menuhin
4,181,Xis
5,211,Wilhelm Kempff
6,154,Whitesnake
7,73,Vinícius E Qurteto Em Cy
8,74,Vinícius E Odette Lara
9,71,Vinícius De Moraes & Baden Powell


### 1.3) Write a query to return all the albums in the database from Led Zeppelin.

In [8]:
# CodeGrade step1.3
# Replace None with appropriate code
# Hint: Use the Artist and Album tables!

second_query = """

SELECT Al.Title
FROM Album Al
JOIN Artist Ar
ON Ar.ArtistId = Al.ArtistId
WHERE Ar.Name = 'Led Zeppelin'
    
"""

pd.read_sql(second_query, conn)

,Title
0,BBC Sessions [Disc 1] [Live]
1,Physical Graffiti [Disc 1]
2,BBC Sessions [Disc 2] [Live]
3,Coda
4,Houses Of The Holy
5,In Through The Out Door
6,IV
7,Led Zeppelin I
8,Led Zeppelin II
9,Led Zeppelin III


### 1.4) Write a query to return both the artist with the most albums in the database and the number of albums.

In [104]:
# CodeGrade step1.4
# Replace None with appropriate code

third_query = """
SELECT Ar.Name, COUNT(*) ctr
FROM Album Al
JOIN Artist Ar
ON Ar.ArtistId = Al.ArtistId
GROUP BY Ar.Name
---GROUP BY Ar.ArtistId
ORDER BY ctr DESC
LIMIT 1
"""

pd.read_sql(third_query, conn)

,Name,ctr
0,Iron Maiden,21


In [109]:
third_query_cte = """
WITH AlbumCount AS (
    SELECT ArtistId, COUNT(*) as Count
    FROM Album
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 1
)

SELECT ar.Name, ac.Count
FROM AlbumCount ac
JOIN Artist Ar
ON Ar.ArtistId = ac.ArtistId
GROUP BY Ar.Name
ORDER BY Count DESC
---LIMIT 1
"""

pd.read_sql(third_query_cte, conn)

,Name,Count
0,Iron Maiden,21


In [114]:
third_query_subquery = """

SELECT
    ar.Name, 
    (SELECT COUNT(*) FROM Album GROUP BY ArtistID ORDER BY 1 DESC LIMIT 1) AS Count
FROM
    Artist ar
WHERE
    ar.ArtistId in (SELECT ArtistId FROM Album GROUP BY ArtistId ORDER BY COUNT(*) DESC LIMIT 1)



"""

pd.read_sql(third_query_subquery, conn)

,Name,Count
0,Iron Maiden,21


---
## Part 2: Bayesian Statistics [Suggested time: 15 minutes]
---

A medical test is designed to diagnose a certain disease. The test has a false positive rate of 10%, meaning that 10% of people without the disease will get a positive test result. The test has a false negative rate of 2%, meaning that 2% of people with the disease will get a negative result. Only 1% of the population has this disease.

### 2.1) Create a numeric variable `p_pos_test` containing the probability of a person receiving a positive test result.

Assume that the person being tested is randomly selected from the broader population.

In [46]:
# CodeGrade step2.1
# Replace None with appropriate code
    
false_pos_rate = 0.1
false_neg_rate = 0.02
population_rate = 0.01

p_pos_test = (1 - false_neg_rate)*population_rate + false_pos_rate*(1 - population_rate)

In [48]:
p_pos_test

0.10880000000000001

In [74]:
(.01 * .98) + (.99 * .1)

0.10880000000000001

### 2.2) Create a numeric variable `p_disease_given_pos` containing the probability of a person actually having the disease if they receive a positive test result.

Assume that the person being tested is randomly selected from the broader population.

Hint: Use your answer to the previous question to help answer this one.

In [51]:
# CodeGrade step2.2
# Replace None with appropriate code
    
false_pos_rate = 0.1
false_neg_rate = 0.02
population_rate = 0.01

p_disease_given_pos = (1 - false_neg_rate) * (population_rate) / p_pos_test

In [52]:
p_disease_given_pos

0.09007352941176469

In [79]:
(.98 * .01) / ((.01 * .98) + (.99 * .1))

0.09007352941176469

---
## Part 3: Normal Distribution [Suggested time: 20 minutes]
---
In this part, you will analyze check totals at a TexMex restaurant. We know that the population distribution of check totals for the TexMex restaurant is normally distributed with a mean of \\$20 and a standard deviation of \\$3. 

### 3.1) Create a numeric variable `z_score_26` containing the z-score for a \\$26 check. 

In [55]:
# CodeGrade step3.1
# Replace None with appropriate code

z_score_26 =  (26-20)/3

In [56]:
z_score_26

2.0

### 3.2) Create a numeric variable `p_under_26` containing the approximate proportion of all checks that are less than \\$26.

Hint: Use the answer from the previous question along with the empirical rule, a Python function, or this [z-table](https://www.math.arizona.edu/~rsims/ma464/standardnormaltable.pdf).

In [61]:
# Using Python function
p_under_26 = stats.norm.cdf(2.0)
p_under_26

0.9772498680518208

In [60]:
# Using z-table
p_under_26 = 0.97725

In [59]:
# Setting the distribution parameters.
p_under_26 = stats.norm(loc=20, scale=3).cdf(26)
p_under_26

0.9772498680518208

### 3.3) Create numeric variables `conf_low` and `conf_high` containing the lower and upper bounds (respectively) of a 95% confidence interval for the mean of one waiter's check amounts using the information below. 

One week, a waiter gets 100 checks with a mean of \\$19 and a standard deviation of \\$3.

In [63]:
# CodeGrade step3.3
# Replace None with appropriate code

n = 100
mean = 19
std = 3

# Calculate standard error
sterr = std/(n**0.5)

# 95% confidence interval has z-score of +- 1.96
conf_low = mean - 1.96*sterr
conf_high = mean + 1.96*sterr
print('The lower bound of the 95% confidence interval is {}'.format(conf_low))
print('The upper bound of the 95% confidence interval is {}'.format(conf_high))

The lower bound of the 95% confidence interval is 18.412
The upper bound of the 95% confidence interval is 19.588


In [115]:
# Using statsmodels stats.norm.interval

conf = stats.norm.interval(alpha=.95, loc=19, scale=sterr)
print('The lower bound of the 95% confidence interval is {}'.format(conf[0]))
print('The upper bound of the 95% confidence interval is {}'.format(conf[1]))

The lower bound of the 95% confidence interval is 18.412010804637983
The upper bound of the 95% confidence interval is 19.587989195362017


### 3.4) Short Answer: Interpret the 95% confidence interval you just calculated in Question 1.3.

In [ ]:
# Your answer here

# A 95% confidence interval means that we are 95% confident that the interval contains the true population mean.
# In this case, that means we are 95% confident that the true mean of this waiter's checks is between
# $18.41 and $19.59

---
## Part 4: Statistical Testing [Suggested time: 20 minutes]
---
The TexMex restaurant recently introduced queso to its menu.

We have a random sample containing 2000 check totals, all from different customers: 1000 check totals for orders without queso ("no queso") and 1000 check totals for orders with queso ("queso").

In the cell below, we load the sample data for you into the arrays `no_queso` and `queso` for the "no queso" and "queso" order check totals, respectively.

In [70]:
# Run this cell without changes

# Load the sample data 
no_queso = pickle.load(open('./no_queso.pkl', 'rb'))
queso = pickle.load(open('./queso.pkl', 'rb'))

### 4.1) Short Answer: State null and alternative hypotheses to use for testing whether customers who order queso spend different amounts of money from customers who do not order queso.

In [ ]:
# Your answer here

# Null hypothesis: Customers who order queso spend the same as those who do not order queso. 

# Alternative hypothesis: Customers who order queso do not spend the same as those who do not order queso.


### 4.2) Short Answer: What would it mean to make a Type I error for this specific hypothesis test?

Your answer should be _specific to this context,_  not a general statement of what Type I error is.

In [ ]:
# Your answer here

# Type I Error (Rejecting the null hypothesis given that it's true): 
# Concluding that queso customers' check amounts are different from 
# non-queso customers' total check amounts when they are actually the same.

### 4.3) Create a numeric variable `p_value` containing the p-value associated with a statistical test of your hypotheses. 

You must identify and implement the correct statistical test for this scenario. You can assume the two samples have equal variances.

Hint: Use `scipy.stats` to calculate the answer - it has already been imported as `stats`. Relevant documentation can be found [here](https://docs.scipy.org/doc/scipy/reference/stats.html#statistical-tests).

In [72]:
# CodeGrade step4.3
# Replace None with appropriate code

p_value = stats.ttest_ind(no_queso, queso).pvalue
p_value

1.29670967092511e-307

### 4.4) Short Answer: Can you reject the null hypothesis using a significance level of $\alpha$ = 0.05? Explain why or why not.

In [ ]:
# Your answer here

# There is enough evidence to reject the null hypothesis at a significance level 
# of alpha = 0.05. This is because the p-value is smaller than 0.05.